### Save and Retrive Data from Redis
- https://redis.com/

In [6]:
import numpy as np
import pandas as pd
import redis

In [7]:
# Connect to Redis Client
# redis-18692.c57.us-east-1-4.ec2.cloud.redislabs.com:18692
# wCneC8tYJRuw0ovU9Kacbr9kgY4UmQdb
hostname = 'redis-19897.c326.us-east-1-3.ec2.cloud.redislabs.com'
portnumber = 19897
password = '6q1uDG9nix1r9mqYG8IrigfgA1BdvJnU'

r = redis.StrictRedis(host=hostname,
                      port=portnumber,
                      password=password)


In [8]:
r.ping()

True

In [9]:
#r.keys()

#### Load the data (numpy zip)

In [10]:
np_file = np.load('dataframe_students_teacher.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values,columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291..."
1,Barack Obama,Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965..."
2,Chris Evans,Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073..."
3,Morgan Freeman,Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788..."
4,Scarlett Johansson,Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967..."


In [11]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967...",Scarlett Johansson@Student


In [12]:
records = df[['name_role','Facial_Features']].to_dict(orient='records')
len(records)

5

## Save data in Redis
- redis hashes
- keyname = 'academy:register'

In [13]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']
    
    # convert numpy array into bytes
    vector_bytes = vector.tobytes()
    
    # save data into redis cloud
    r.hset(name='academy:register',key=name_role,value=vector_bytes)

In [14]:
vector.dtype

dtype('float32')

### Retrive Records from Redis

In [15]:
retrive_dict = r.hgetall(name='academy:register')
retrive_series = pd.Series(retrive_dict)
retrive_series

b'Morgan Freeman@Teacher'        b'\xee\x7f\xf8>\x87\x87\x1e\xbfja\xd0\xbe\xbe\...
b'Barack Obama@Teacher'          b'5\xd6r?\xa5\xf3\x0e\xbf]1\xc2=\xde\xcb\xa3\x...
b'Angelina Jolie@Student'        b'3f\xdd>\xae\x94\x89?\xeb\xb5\x03\xbeN\xa9\xe...
b'Chris Evans@Student'           b')]8?\x05\xf5\xbb\xbf83G\xbf\xcc\xe4\xba\xbf\...
b'Scarlett Johansson@Student'    b'K\xe9C>\x8d\x14\xfa\xbe\x8eY\xd8\xbf\xd3\x81...
dtype: object

In [16]:
#np.frombuffer(retrive_series[b'Morgan Freeman@Teacher'],dtype=np.float32).shape
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))

In [17]:
retrive_series.index = index

In [18]:
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df

,name_role,facial_features
0,Morgan Freeman@Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788..."
1,Barack Obama@Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965..."
2,Angelina Jolie@Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291..."
3,Chris Evans@Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073..."
4,Scarlett Johansson@Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967..."


### Registration Form

In [19]:
import cv2
from insightface.app import FaceAnalysis

In [20]:
# configure face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='insightface_model',
                       providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)
# warning: don't set det_thresh < 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


##### Step-1: Collect Person and Role

In [21]:
# python input
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role = input("""
    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
    """)
    
    if role in ('1','2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        
        break
    else:
        print('Invalid entry try again')
        if i == 3:
            print('Exceeds maximum trails')
            
key = person_name +'@'+ role
print('Your name = ', person_name)
print('Your role =', role)
print('Key =',key)

Your name =  Aman Kumar Jha
Your role = Student
Key = Aman Kumar Jha@Student


##### Step-2: Collect Samples

In [22]:
cap = cv2.VideoCapture(0) # 0 default camera, 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    
    # get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1),(x2,y2),(0,255,0),1)
        
        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
        
    if sample >= 200: 
        break
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'): # this is trigger only when I press letter q (lowercase q) in my keyboard
        break
        
cap.release()
cv2.destroyAllWindows()

In [23]:
len(face_embeddings)

200

##### Step-3: Optimize Data & Save in Redis Database

In [24]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [25]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [26]:
# save key and facial bytes into redis database
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1